In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

In [2]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

C:\Users\kaues\AppData\Local\Temp\ipykernel_7144\3558521049.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\kaues\AppData\Local\Temp\ipykernel_7144\3558521049.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain = ConversationChain(


In [3]:
chain.predict(input="olá")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: olá
AI:

> Finished chain.


' Olá! Como posso te ajudar hoje?'

In [4]:
chain.run("olá")

C:\Users\kaues\AppData\Local\Temp\ipykernel_7144\3150572815.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("olá")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: olá
AI:  Olá! Como posso te ajudar hoje?
Human: olá
AI:

> Finished chain.


'Olá novamente! Estou aqui para responder a qualquer pergunta que você possa ter. Como posso ajudar você hoje?'

In [5]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Essa é uma conversa amigável entre um humano e uma IA

Conversa atual:
{history}
Human: {input}
AI:"""
)

In [6]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
memory = ConversationBufferMemory()
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

In [7]:
chain.predict(input="Oi")



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA

Conversa atual:

Human: Oi
AI:

> Finished chain.


'Olá! Como posso ajudar você hoje?'

### LLMChain

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.llm import LLMChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [9]:
prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)

chain = LLMChain(llm=chat, prompt=prompt)
produto = "LLMs com LangChain"
chain.run(produto)

C:\Users\kaues\AppData\Local\Temp\ipykernel_7144\1676720191.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat, prompt=prompt)


'1. LangTech Solutions\n2. ChainLink Solutions\n3. LLM Innovations\n4. LangChain Development\n5. LinkChain Dynamics'

### SimpleSequentialChain

In [10]:
from langchain.chains import SimpleSequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [12]:
prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt)

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com nome {nome_empresa}.
    Informe uma descrição de até 20 palavras.
    """
)

chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao)


In [13]:
chain = SimpleSequentialChain(
    chains=[chain_nome, chain_descricao],
    verbose=True
)

chain.run(input=produto)



> Entering new SimpleSequentialChain chain...
"LangChain Solutions"
A LangChain Solutions é uma empresa especializada em soluções de tecnologia da informação para otimização de processos empresariais.

> Finished chain.


'A LangChain Solutions é uma empresa especializada em soluções de tecnologia da informação para otimização de processos empresariais.'

### SequentialChain

In [14]:
from langchain.chains import SequentialChain

chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [15]:
prompt = PromptTemplate.from_template(
    """
    Qual um bom nome de empresa que desenvolve soluções em {produto}?
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com o nome {nome_empresa} e que 
    produz seguinte {produto}.
    Informe uma descrição de até 30 palavras de atividades dessa empresa
    """
)
chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao, 
                           output_key="descricao_empresa")

prompt_traducao = PromptTemplate.from_template(
    """
    Crie um nome em espanhol para a empresa de nome {nome_empresa}
    que possui a seguinte descrição {descricao_empresa}
    """
)
chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, 
                          output_key="nome_espanhol")

In [16]:
chain = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=["produto"],
    output_variables=["nome_empresa", "descricao_empresa", "nome_espanhol"],
    verbose=True
)

resposta = chain.invoke({"produto": produto})



> Entering new SequentialChain chain...

> Finished chain.


In [17]:
resposta["produto"]

'LLMs com LangChain'

In [18]:
resposta["nome_empresa"]

'LangChain Solutions'

In [19]:
resposta["descricao_empresa"]

'A LangChain Solutions é especializada na produção de soluções de tecnologia de idiomas e oferece LLMs (Language Learning Machines) com a plataforma LangChain.'

In [20]:
resposta["nome_espanhol"]

'Soluciones de Idiomas LangChain'